In [1]:
import gym
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch import optim
from torch.nn import functional as F

In [2]:
env=gym.make('FrozenLake-v0')

D:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(16,64)
        self.fc2=nn.Linear(64,64)
        self.fc3=nn.Linear(64,96)
        self.fc4=nn.Linear(96,96)
        self.fc5=nn.Linear(96,64)
        self.fc6=nn.Linear(64,64)
        self.fc7=nn.Linear(64,4)
    def forward(self,x):
        x=Variable(x)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=F.relu(self.fc4(x))
        x=F.relu(self.fc5(x))
        x=F.relu(self.fc6(x))
        x=self.fc7(x)
        return x
model=Net()

In [5]:
def onehot2tensor(state):
    tmp=np.zeros(16)
    tmp[state]=1
    vector=np.array(tmp,dtype='float32')
    tensor=torch.from_numpy(vector).float()
    return tensor

def applymodel(tensor):
    output_tensor=model(tensor)
    output_array=output_tensor.data.numpy()
    return output_tensor,output_array

In [6]:
total_reward=0.0
criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [11]:
for i_episode in range(1000):
    observation=env.reset()
    episode_reward=0.0
    total_loss=0.0
    for t in range(100):
        #env.render()
        current_state=observation
        optimizer.zero_grad()
        current_tensor=onehot2tensor(current_state)
        current_output_tensor,current_output_array=applymodel(current_tensor)
        if np.random.rand()<0.1:
            action=env.action_space.sample()
        else:
            action=np.argmax(current_output_array)
        
        observation,reward,done,info=env.step(action)
        observation_tensor=onehot2tensor(observation)
        observation_output_tensor,observation_output_array=applymodel(observation_tensor)
        
        q=reward+0.99*np.max(observation_output_array)
        q_array=np.copy(current_output_array)
        q_array[action]=q
        q_variable=Variable(torch.Tensor(q_array))
        
        loss=criterion(current_output_tensor,q_variable)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        
        if done:
            episode_reward+=reward
    total_reward+=episode_reward
    
    if (i_episode+1)%100==0:
        print(i_episode+1,total_loss,total_reward)

100 1.169472501505453e-05 4.0
200 1.1130832795364976e-25 4.0
300 2.75308558517853e-23 4.0
400 1.5092777934988533e-05 8.0
500 8.71338798980048e-06 9.0
600 1.1366358558301173e-11 12.0
700 3.022720612891921e-11 14.0
800 2.0730112055389327e-05 18.0
900 5.054880649037278e-09 20.0
1000 7.936006698335805e-10 21.0
